### SentinelForge

#### This is the start of training file for SentinelForge

1. Import necessary library

In [29]:
import os


2. Data pipeline

In [50]:
current_dir = os.getcwd()

data_path = os.path.join(current_dir, '..\data')

data_path_filtered = [file for file in os.listdir(data_path) if file.endswith((".c", ".cpp"))]
print(data_path_filtered)

['main.c', 'main.cpp', 'main_0.c', 'main_1.c', 'main_10.c', 'main_100.c', 'main_101.c', 'main_102.c', 'main_103.c', 'main_104.c', 'main_105.c', 'main_106.c', 'main_107.c', 'main_108.c', 'main_109.c', 'main_11.c', 'main_110.c', 'main_111.c', 'main_112.c', 'main_113.c', 'main_114.c', 'main_115.c', 'main_116.c', 'main_117.c', 'main_118.c', 'main_119.c', 'main_12.c', 'main_120.c', 'main_121.c', 'main_122.c', 'main_123.c', 'main_124.c', 'main_125.c', 'main_126.c', 'main_127.c', 'main_128.c', 'main_129.c', 'main_13.c', 'main_130.c', 'main_131.c', 'main_132.c', 'main_133.c', 'main_134.c', 'main_135.c', 'main_136.c', 'main_137.c', 'main_138.c', 'main_139.c', 'main_14.c', 'main_140.c', 'main_141.c', 'main_142.c', 'main_143.c', 'main_144.c', 'main_145.c', 'main_146.c', 'main_147.c', 'main_148.c', 'main_149.c', 'main_15.c', 'main_150.c', 'main_151.c', 'main_152.c', 'main_153.c', 'main_154.c', 'main_155.c', 'main_156.c', 'main_157.c', 'main_158.c', 'main_159.c', 'main_16.c', 'main_160.c', 'main_16

Test opening one file to check on its content

In [51]:
file_path = os.path.join(data_path,data_path_filtered[0])

# Read the file content into a string
with open(file_path, "r", encoding="utf-8") as f:
    code = f.read()

# Remove preprocessor directives (lines starting with #)
filtered_code = "\n".join(line for line in code.splitlines() if not line.strip().startswith("#"))

# Parse the code using pycparser
parser = CParser()
ast = parser.parse(filtered_code)

# Display the AST
ast.show()

FileAST: 
  FuncDef: 
    Decl: main, [], [], [], []
      FuncDecl: 
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound: 
      Decl: string, [], [], [], []
        PtrDecl: []
          TypeDecl: string, [], None
            IdentifierType: ['char']
      Assignment: =
        ID: string
        Cast: 
          Typename: None, [], None
            PtrDecl: []
              TypeDecl: None, [], None
                IdentifierType: ['char']
          FuncCall: 
            ID: malloc
            ExprList: 
              BinaryOp: *
                UnaryOp: sizeof
                  Typename: None, [], None
                    TypeDecl: None, [], None
                      IdentifierType: ['char']
                Constant: int, 1000


Supportive C Parser function to generate AST for each C file

In [52]:
from pycparser import CParser

def parse_c_code_to_ast(code):
    filtered_code = "\n".join(line for line in code.splitlines() if not line.strip().startswith("#"))

# Parse the code using pycparser
    parser = CParser()
    ast = parser.parse(filtered_code)
    return ast

In [101]:
import pycparser


def extract_features_from_ast(ast):
    features = {}

    def traverse_ast(node, depth=0):
        if node is not None:
            node_type = type(node).__name__

            # Feature 1: Track the count of node types
            # if node_type not in features:
            #     features[node_type] = 0
            # features[node_type] += 1

            # Feature 2: Extract details of variable declarations (Decl) and function declarations (FuncDecl)
            if node_type == 'Decl':
                var_name = node.name  # Get the variable name
                var_type = None

                # If it's a simple type (e.g., 'char', 'int')
                if isinstance(node.type, pycparser.c_ast.TypeDecl):
                    var_type = node.type.type.names[0] if node.type.type else None
                
                # If it's a pointer type (e.g., 'char*')
                if isinstance(node.type, pycparser.c_ast.PtrDecl):
                    # Traverse the underlying TypeDecl to get the base type
                    if isinstance(node.type.type, pycparser.c_ast.TypeDecl):
                        base_type = node.type.type.type.names[0] if node.type.type.type else None
                        var_type = f'pointer to {base_type}'
                
                # If it's an array type (e.g., 'char[]')
                if isinstance(node.type, pycparser.c_ast.ArrayDecl):
                    # Traverse the underlying TypeDecl to get the base type
                    if isinstance(node.type.type, pycparser.c_ast.TypeDecl):
                        base_type = node.type.type.type.names[0] if node.type.type.type else None
                        var_type = f'array to {base_type}'
                    
                    # Extract initialization values (if any)
                    if isinstance(node.init, pycparser.c_ast.InitList):
                        features[f'{var_name}_init'] = "Yes"
                    else:
                        features[f'{var_name}_init'] = "No"

                # Add the variable and its type to the list
                if var_type:
                    features['string'] = var_type



            if node_type == 'FuncDef':
                # Extract the function name
                func_name = node.decl.name
                features[f'func_{func_name}_defined'] = 1  # To track the presence of the function definition
                # Extract function's return type and parameters
                func_return_type = None
                if isinstance(node.decl.type, pycparser.c_ast.FuncDecl):
                    return_type = node.decl.type.type
                    if isinstance(return_type, pycparser.c_ast.TypeDecl):
                        func_return_type = return_type.type.names if return_type.type else None
                #features[f'func_{func_name}_return_type'] = func_return_type

            # Feature 3: Extract function calls like malloc
            if node_type == 'FuncCall':
                func_name = node.name.name  # Function name (e.g., malloc)
                if func_name == 'malloc':
                    features['malloc_used'] = 1  # Track if malloc is used

            # Recursively visit child nodes
            for field, child in node.children():
                traverse_ast(child, depth + 1)

    traverse_ast(ast)
    return features


2.1 Test functions

In [102]:
file_path = os.path.join(data_path,data_path_filtered[0])
print(file_path)

# Read the file content into a string
with open(file_path, "r", encoding="utf-8") as f:
    code = f.read()

ast = parse_c_code_to_ast(code)
features = extract_features_from_ast(ast)

ast.show()
print(features)

c:\Users\TRUNG\Desktop\file_backup_dual_boot\Coding\1.C++\iNTUition_dummy\model_training\..\data\main.c
FileAST: 
  FuncDef: 
    Decl: main, [], [], [], []
      FuncDecl: 
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound: 
      Decl: string, [], [], [], []
        PtrDecl: []
          TypeDecl: string, [], None
            IdentifierType: ['char']
      Assignment: =
        ID: string
        Cast: 
          Typename: None, [], None
            PtrDecl: []
              TypeDecl: None, [], None
                IdentifierType: ['char']
          FuncCall: 
            ID: malloc
            ExprList: 
              BinaryOp: *
                UnaryOp: sizeof
                  Typename: None, [], None
                    TypeDecl: None, [], None
                      IdentifierType: ['char']
                Constant: int, 1000
{'func_main_defined': 1, 'string': 'pointer to char', 'malloc_used': 1}


In [103]:
file_path = os.path.join(data_path, 'main_only_p.c')
print(file_path)

# Read the file content into a string
with open(file_path, "r", encoding="utf-8") as f:
    code = f.read()

ast = parse_c_code_to_ast(code)
features = extract_features_from_ast(ast)

ast.show()
print(features)

c:\Users\TRUNG\Desktop\file_backup_dual_boot\Coding\1.C++\iNTUition_dummy\model_training\..\data\main_only_p.c
FileAST: 
  FuncDef: 
    Decl: main, [], [], [], []
      FuncDecl: 
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound: 
      Decl: string, [], [], [], []
        PtrDecl: []
          TypeDecl: string, [], None
            IdentifierType: ['char']
{'func_main_defined': 1, 'string': 'pointer to char'}


In [104]:
file_path = os.path.join(data_path, 'main_init.cpp')
print(file_path)

# Read the file content into a string
with open(file_path, "r", encoding="utf-8") as f:
    code = f.read()

ast = parse_c_code_to_ast(code)
features = extract_features_from_ast(ast)

ast.show()
print(features)

c:\Users\TRUNG\Desktop\file_backup_dual_boot\Coding\1.C++\iNTUition_dummy\model_training\..\data\main_init.cpp
FileAST: 
  FuncDef: 
    Decl: main, [], [], [], []
      FuncDecl: 
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound: 
      Decl: string, [], [], [], []
        ArrayDecl: []
          TypeDecl: string, [], None
            IdentifierType: ['char']
          Constant: int, 1000
{'func_main_defined': 1, 'string_init': 'No', 'string': 'array to char'}


In [105]:
file_path = os.path.join(data_path,data_path_filtered[1])
print(file_path)

# Read the file content into a string
with open(file_path, "r", encoding="utf-8") as f:
    code = f.read()

ast = parse_c_code_to_ast(code)
features = extract_features_from_ast(ast)

ast.show()
print(features)

c:\Users\TRUNG\Desktop\file_backup_dual_boot\Coding\1.C++\iNTUition_dummy\model_training\..\data\main.cpp
FileAST: 
  FuncDef: 
    Decl: main, [], [], [], []
      FuncDecl: 
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound: 
      Decl: string, [], [], [], []
        ArrayDecl: []
          TypeDecl: string, [], None
            IdentifierType: ['char']
          Constant: int, 1000
        InitList: 
          Constant: char, 'h'
          Constant: char, 'e'
          Constant: char, 'l'
          Constant: char, 'l'
          Constant: char, 'o'
{'func_main_defined': 1, 'string_init': 'Yes', 'string': 'array to char'}


2.2 Data Loading

In [46]:
import json
import os
from pycparser import parse_file

# Load the JSON file with labels
with open('data.json', 'r') as f:
    labeled_data = json.load(f)

# Prepare the dataset
X = []  # Feature matrix
y = []  # Labels

# Class labels (you can map your error types to numerical values)
label_map = {
    "UNALLOCATED_POINTER": 0,
    "MEMORY_NOT_FREE": 1,
    "UNUSED_VARIABLE": 2,
    "NONE": 3,
    "OTHERS": 4
}

# Loop through each file and process it
data_path = data_path_filtered
for entry in labeled_data:
    file_name = entry["file"]
    error_type = entry["error"]
    
    # Map the error type to a class label
    label = label_map.get(error_type, 4)  # Default to 'Other error' if not found
    
    # Read the C code from the file
    with open(file_name, 'r') as f:
        c_code = f.read()

    # Parse the C code into an AST
    try:
        ast = parse_c_code_to_ast(c_code)
        features = extract_features_from_ast(ast)
        X.append(features)  # Add features to the dataset
        y.append(label)  # Add label to the target vector
    except Exception as e:
        print(f"Error parsing {file_name}: {e}")
        continue  # Skip files that can't be parsed


3. Training

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


TypeError: float() argument must be a string or a real number, not 'dict'